IMPORTING ALL REQUIRED MODULES

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

PREPROCESSING THE DATA

In [2]:
# Load the training data
data = pd.read_csv('train.csv')

# Fill missing values
data.fillna(method='ffill', inplace=True)

# Define features (excluding lag features)
features = data[['year', 'month', 'day', 'latitude', 'longitude', 'zon.winds', 'mer.winds', 'humidity', 'air temp.']]
target = data['s.s.temp.']

# Reshape input to be [samples, time steps, features]
X = np.reshape(features.values, (features.shape[0], 1, features.shape[1]))
y = target.values

C:\Users\utkar\AppData\Local\Temp\ipykernel_9652\472559042.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


SPLITTING THE DATA INTO TRAIN & TEST SETS

In [3]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

DEFINING AN LSTM MODEL WITH ONE LSTM LAYER FOLLOWED BY A DENSE LAYER

In [4]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

c:\Users\utkar\OneDrive\Desktop\For TOTW\totw\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


TRAINING THE MODEL ON THE TRAIN DATASET

In [5]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 2.3931 - val_loss: 1.5246
Epoch 2/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.4279 - val_loss: 1.0965
Epoch 3/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.0778 - val_loss: 0.8696
Epoch 4/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9248 - val_loss: 1.0613
Epoch 5/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.8934 - val_loss: 1.2767
Epoch 6/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.8832 - val_loss: 0.7843
Epoch 7/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.8358 - val_loss: 1.5888
Epoch 8/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.8352 - val_loss: 0.8152
Epoch 9/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.8039 - val_loss: 0.7777
Epoch 10/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.8238 - val_loss: 0.7595
Epoch 11/50
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.7820 - val_loss: 0.8794
Epoch 12/50
2611/2611 ━━━━━━━

SAVING THE MODEL

In [6]:
# Save the model
model.save('sea_surface_temp_model.keras')

CHECKING THE ACCURACY OF THE MODEL ON THE SPLITTED TEXT DATSET (X_val) USING MAPE

In [10]:
import numpy as np
from tensorflow.keras.models import load_model

# Assuming X_val_reshaped and y_val are already defined from the train_test_split step
# You can load them if they were saved or recompute them using the train_test_split

# Load the saved model
model = load_model('sea_surface_temp_model.keras')

# Predict using the validation data
predictions = model.predict(X_val)

# Define a function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate the MAPE
mape = mean_absolute_percentage_error(y_val, predictions)

print(f"Model Accuracy (MAPE): {100 - mape:.2f}%")

653/653 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Model Accuracy (MAPE): 91.34%


C:\Users\utkar\AppData\Local\Temp\ipykernel_9652\71541620.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_data.fillna(method='ffill', inplace=True)


3263/3263 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Predictions saved to predicted_sea_surface_temp.csv
